In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -uq "/content/drive/MyDrive/train.zip" -d "/content/drive/My Drive/train"
!unzip -uq "/content/drive/MyDrive/test.zip" -d "/content/drive/My Drive/test"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
import os
import glob as gb
import cv2
import tensorflow as tf
import keras
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
from sklearn.model_selection import train_test_split


In [ ]:
trainpath = '../content/drive/MyDrive/train/seg_train/'
testpath = '../content/drive/MyDrive/test/test/'

In [ ]:
for folder in  os.listdir(trainpath) : 
    files = gb.glob(pathname= str( trainpath + folder + '/*.jpg'))
    print(f'In {folder}, found {len(files)} pics ')

In glacier, found 2404 pics 
In forest, found 2271 pics 
In street, found 2382 pics 
In mountain, found 2512 pics 
In sea, found 2274 pics 
In buildings, found 2191 pics 


In [ ]:
code = {'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}

def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x    

In [ ]:
X_train = []
Y_train = []
for folder in  os.listdir(trainpath) : 
    files = gb.glob(pathname= str( trainpath + folder + '/*.jpg'))
    for file in files: 
        image = cv2.imread(file)
        image_array = cv2.resize(image , (150,150))
        X_train.append(list(image_array))
        Y_train.append(code[folder])

In [ ]:
X_pred = []
files = gb.glob(pathname= str(testpath + '/*.jpg'))
for file in files: 
    image = cv2.imread(file)
    #image_array = cv2.resize(image , (150,150))
    X_pred.append(list(image_array))      

In [ ]:
from tensorflow.keras.utils import load_img

X = []
for i in range(0, 3000):
  image = load_img(f'../content/drive/MyDrive/test/test/{i}.jpg')
  X.append(image) 
for j in range(len(X)):
  X[j] = np.array(X[j])
  X[j] = cv2.resize(X[j] , (150,150))

In [ ]:
X = np.array(X)
X.shape

(3000, 150, 150, 3)

In [ ]:
X_pred = np.array(X_pred)
X_pred.shape

(3000, 150, 150, 3)

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
model = keras.models.Sequential([

 keras.layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)),
 keras.layers.Conv2D(180,kernel_size=(3,3),activation='relu'),
 keras.layers.MaxPool2D(5,5),
 keras.layers.Conv2D(180,kernel_size=(3,3),activation='relu'),
 keras.layers.Conv2D(140,kernel_size=(3,3),activation='relu'),
 keras.layers.Conv2D(100,kernel_size=(3,3),activation='relu'),
 keras.layers.Conv2D(50,kernel_size=(3,3),activation='relu'),
 keras.layers.MaxPool2D(5,5),
 keras.layers.Flatten(),
 keras.layers.Dense(180,activation='relu'),
 keras.layers.Dense(100,activation='relu'),
 keras.layers.Dense(50,activation='relu'),
 keras.layers.Dropout(rate=0.5),
 keras.layers.Dense(6,activation='softmax'),
])

model.compile(optimizer=Optimizer.Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()
#SVG(model_to_dot(model).create(prog='dot', format='svg'))
#Utils.plot_model(model,to_file='model.png',show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 200)     5600      
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 180)     324180    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 29, 180)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 180)       291780    
                                                                 
 conv2d_3 (Conv2D)           (None, 25, 25, 140)       226940    
                                                                 
 conv2d_4 (Conv2D)           (None, 23, 23, 100)       126100    
                                                        

In [ ]:
Trained = model.fit(X_train, Y_train, epochs=50, batch_size=64, verbose=1)

Epoch 1/50
220/220 [==============================] - 144s 653ms/step - loss: 1.5166 - accuracy: 0.3966
Epoch 2/50
220/220 [==============================] - 143s 648ms/step - loss: 1.1876 - accuracy: 0.5444
Epoch 3/50
220/220 [==============================] - 142s 647ms/step - loss: 1.0183 - accuracy: 0.6300
Epoch 4/50
220/220 [==============================] - 142s 647ms/step - loss: 0.9171 - accuracy: 0.6794
Epoch 5/50
220/220 [==============================] - 142s 648ms/step - loss: 0.8057 - accuracy: 0.7229
Epoch 6/50
220/220 [==============================] - 142s 648ms/step - loss: 0.7449 - accuracy: 0.7475
Epoch 7/50
220/220 [==============================] - 142s 647ms/step - loss: 0.6890 - accuracy: 0.7712
Epoch 8/50
220/220 [==============================] - 143s 648ms/step - loss: 0.6445 - accuracy: 0.7887
Epoch 9/50
220/220 [==============================] - 143s 648ms/step - loss: 0.5995 - accuracy: 0.8008
Epoch 10/50
220/220 [==============================] - 142s 648m

In [ ]:
predictions = model.predict(X)
predictions.shape

94/94 [==============================] - 11s 97ms/step


(3000, 6)

In [ ]:
preds_index = np.argmax(predictions, axis=1)
preds_index

array([5, 4, 0, ..., 0, 1, 1])

In [ ]:
df = pd.read_csv("../content/drive/MyDrive/sample (1).csv")

In [ ]:
X=df['id']
X.shape

(3000,)

In [ ]:
preds_index.reshape(-1,1).shape

(3000, 1)

In [ ]:
result=pd.DataFrame({"id":X,"label":preds_index})
result

,id,label
0,0,5
1,1,4
2,2,0
3,3,3
4,4,4
...,...,...
2995,2995,5
2996,2996,4
2997,2997,0
2998,2998,1


In [ ]:
result.to_csv("Result2.csv",index=False)